# Practical Exercise: Data Preparation and Visualisation

## Exercise I. Data Loading and Preparation

In [8]:
import pandas as pd
import numpy as np
import h5py
import os, glob
import re
from datetime import datetime, date, time
from six import iteritems

In blond.py file, you can find the Blond class which handles data input, output and calibration.

In [9]:
from blond import Blond

In [11]:
def get_time_diff(t1,t2):
    t1_s = (t1.hour*60*60 + t1.minute*60 + t1.second)
    t2_s = (t2.hour*60*60 + t2.minute*60 + t2.second)

    delta_s = max([t1_s, t2_s]) - min([t1_s, t2_s])
    return delta_s

###  1. Data Reading and Exploration

### Reading files

In [3]:
blond = Blond(date(2016,10,5))

""" Define a timeframe"""
start_ts = time(0,0,50) # start_hours_minutes
end_ts   = time(1,20,55)

"""Read MEDAL and CLEAR data """
blond.read_files(start_ts, end_ts)


"""Checking if files have been retrieved"""
blond.list_files()
    

{'clear': [<HDF5 file "clear-2016-10-05T01-41-56.615220T+0200-0001326.hdf5" (mode r+)>],
 'medal-1': [<HDF5 file "medal-1-2016-10-05T02-02-13.946221T+0200-0000444.hdf5" (mode r+)>],
 'medal-2': [<HDF5 file "medal-2-2016-10-05T02-02-17.632493T+0200-0000444.hdf5" (mode r+)>],
 'medal-3': [<HDF5 file "medal-3-2016-10-05T02-00-09.452005T+0200-0000444.hdf5" (mode r+)>]}

### Exploration


In [4]:
"""signals acquisited by MEDAL"""
medal_file = blond.list_files()['medal-1'][0]
[key for key in medal_file.keys()]

['current1',
 'current2',
 'current3',
 'current4',
 'current5',
 'current6',
 'voltage']

In [5]:
"""signals acquisited by CLEAR"""
clear_file = blond.list_files()['clear'][0]
[key for key in clear_file.keys()]

['current1', 'current2', 'current3', 'voltage1', 'voltage2', 'voltage3']

### Centering and calibrating

In [6]:
device = 'medal-2'
signal = 'current1'

"""Raw signal with offset and calibration factor attributes"""
dict_signal = blond.dict_read_signal(device, signal)
print(dict_signal)


"""calibrated signal"""
blond.center_and_calibrate(dict_signal)

{'signal': 'medal-2_current1', 'attributes': [{'calibration_factor': 0.015151515, 'values': array([-5, -1, -6, ..., -7, -5, -5], dtype=int16), 'DC_offset': 2500}]}


{'medal-2_current1': array([37.80303 , 37.863636, 37.78788 , ..., 37.772728, 37.80303 ,
        37.80303 ], dtype=float32)}

## Exercise III. Feature Extraction

In order to run the Dash-Plotly application, you need to run "python app.py". After the server runs, you can connect to the server and use the application.

## Exercise IV. Visualization

In [ ]:
import matplotlib.pyplot as plt

sps = 6400
phase = 1
requested_time = time(1,10,10)
file_index = blond.find_corresponding_file(requested_time, blond.time_stamps['clear'])
time_diff = get_time_diff(requested_time , blond.time_stamps['clear'][file_index])
data_index_shift = time_diff * sps
temp_data = blond.list_files()['clear'][file_index]["current"+str(phase)][data_index_shift:data_index_shift + int(sps*0.1)]

curr_rms = []
for curr in temp_data:
    curr_rms.append(np.sqrt(np.mean(curr**2)))
    
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

plot1 = ax[0]
plot1.plot(temp_data, color='b', label='Current')
plot1.plot(curr_rms, color='r', label='Period RMS of Current')
plot1.set_title('Current of AC Electrolux')
plot1.set_xlabel('Time [s]')
plot1.set_ylabel('Current [A]')
plot1.legend()

plot2 = ax[1]
plot2.plot(temp_data, color='b', label='Current')
plot2.plot(curr_rms, color='r', label='Period RMS of Current')
plot2.set_title('Waveform Comparison of AC Electrolux')
plot2.set_xlabel('Time [ms]')
plot2.legend()

plot3 = ax[2]
plot3.plot(temp_data, color='b', label='Current')
plot3.plot(curr_rms, color='r', label='Period RMS of Current')
plot3.set_title('Power over Time of AC Electrolux')
plot3.set_xlabel('Time [s]')
plot3.set_ylabel('Power [W]')
plot3.legend()

plt.show()